In [1]:
from surprise import Dataset
from surprise import Reader
from surprise import SVD
from surprise.model_selection import GridSearchCV
import pandas as pd

In [2]:
df = pd.read_csv('Ratings.csv', low_memory=False)
books = pd.read_csv('Books.csv', low_memory=False)

df.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [3]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [4]:
books = books[['ISBN', 'Book-Title']]
books.head()

,ISBN,Book-Title
0,0195153448,Classical Mythology
1,0002005018,Clara Callan
2,0060973129,Decision in Normandy
3,0374157065,Flu: The Story of the Great Influenza Pandemic...
4,0393045218,The Mummies of Urumchi


In [5]:
df = pd.merge(df, books, on='ISBN')
df

,User-ID,ISBN,Book-Rating,Book-Title
0,276725,034545104X,0,Flesh Tones: A Novel
1,2313,034545104X,5,Flesh Tones: A Novel
2,6543,034545104X,0,Flesh Tones: A Novel
3,8680,034545104X,5,Flesh Tones: A Novel
4,10314,034545104X,9,Flesh Tones: A Novel
...,...,...,...,...
1031131,276688,0517145553,0,Mostly Harmless
1031132,276688,1575660792,7,Gray Matter
1031133,276690,0590907301,0,Triplet Trouble and the Class Trip (Triplet Tr...
1031134,276704,0679752714,0,A Desert of Pure Feeling (Vintage Contemporaries)


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1031136 entries, 0 to 1031135
Data columns (total 4 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1031136 non-null  int64 
 1   ISBN         1031136 non-null  object
 2   Book-Rating  1031136 non-null  int64 
 3   Book-Title   1031136 non-null  object
dtypes: int64(2), object(2)
memory usage: 39.3+ MB


In [7]:
df = df[df['Book-Rating'] != 0]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 383842 entries, 1 to 1031135
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   User-ID      383842 non-null  int64 
 1   ISBN         383842 non-null  object
 2   Book-Rating  383842 non-null  int64 
 3   Book-Title   383842 non-null  object
dtypes: int64(2), object(2)
memory usage: 14.6+ MB


In [8]:
counts = df['User-ID'].value_counts()
df = df[df['User-ID'].isin(counts[counts > 10].index)]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 255819 entries, 1 to 1031132
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   User-ID      255819 non-null  int64 
 1   ISBN         255819 non-null  object
 2   Book-Rating  255819 non-null  int64 
 3   Book-Title   255819 non-null  object
dtypes: int64(2), object(2)
memory usage: 9.8+ MB


In [9]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df['UserID']=le.fit_transform(df['User-ID'])
df['BookID']=le.fit_transform(df['ISBN'])
df.drop('User-ID',axis=1,inplace=True)
df

,ISBN,Book-Rating,Book-Title,UserID,BookID
1,034545104X,5,Flesh Tones: A Novel,46,21549
3,034545104X,5,Flesh Tones: A Novel,165,21549
4,034545104X,9,Flesh Tones: A Novel,199,21549
14,034545104X,8,Flesh Tones: A Novel,1658,21549
18,034545104X,9,Flesh Tones: A Novel,2119,21549
...,...,...,...,...,...
1031120,0140290788,9,Time Out Chicago (Time Out Chicago),5943,8527
1031121,0609806491,7,The Zen of Proposal Writing: An Expert's Stres...,5943,59641
1031123,1931333246,10,Chorus,5943,110132
1031127,0312264186,8,You Can't Catch Death: A Daughter's Memoir,5944,14956


In [10]:
reader = Reader(rating_scale=(1, 10))

data = Dataset.load_from_df(df[['UserID', 'BookID', 'Book-Rating']], reader=reader)

param_grid = {
    "n_epochs": [5, 10],
    "lr_all": [0.002, 0.005],
    "reg_all": [0.4, 0.6]
}
gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=3, n_jobs=-1, joblib_verbose=1)

gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])

In [11]:
svd = SVD(n_epochs = 10, lr_all=0.005, reg_all=0.4)

In [12]:
user = df['UserID'].sample()
user

439506    3226
Name: UserID, dtype: int64

In [13]:
rated = df[df['UserID'].isin(user)]
rated

,ISBN,Book-Rating,Book-Title,UserID,BookID
7276,0316666343,10,The Lovely Bones: A Novel,3226,17952
33644,0679746048,6,"Girl, Interrupted",3226,66830
36983,038550926X,10,The Devil Wears Prada : A Novel,3226,32422
37340,0345443284,5,While I Was Gone,3226,21426
72625,0786868716,10,The Five People You Meet in Heaven,3226,78325
111693,0446605239,10,The Notebook,3226,42955
124021,1551669293,10,204 Rosewood Lane,3226,98585
131678,0446531480,10,A Body to Die For,3226,42745
170104,0399148779,10,Eleventh Hour: An FBI Thriller,3226,35716
170286,0345445694,7,The Goddesses of Kitchen Avenue,3226,21453


In [14]:
recommendations = df[~df['UserID'].isin(user)]
recommendations

,ISBN,Book-Rating,Book-Title,UserID,BookID
1,034545104X,5,Flesh Tones: A Novel,46,21549
3,034545104X,5,Flesh Tones: A Novel,165,21549
4,034545104X,9,Flesh Tones: A Novel,199,21549
14,034545104X,8,Flesh Tones: A Novel,1658,21549
18,034545104X,9,Flesh Tones: A Novel,2119,21549
...,...,...,...,...,...
1031120,0140290788,9,Time Out Chicago (Time Out Chicago),5943,8527
1031121,0609806491,7,The Zen of Proposal Writing: An Expert's Stres...,5943,59641
1031123,1931333246,10,Chorus,5943,110132
1031127,0312264186,8,You Can't Catch Death: A Daughter's Memoir,5944,14956


In [15]:
recommendations = recommendations.drop_duplicates(subset=['BookID'])
recommendations

,ISBN,Book-Rating,Book-Title,UserID,BookID
1,034545104X,5,Flesh Tones: A Novel,46,21549
66,0446520802,10,The Notebook,134,42516
181,2080674722,7,Les Particules Elementaires,4121,111065
189,038550120X,10,A Painted House,221,32359
370,0425115801,10,Lightning,45,36898
...,...,...,...,...,...
1031120,0140290788,9,Time Out Chicago (Time Out Chicago),5943,8527
1031121,0609806491,7,The Zen of Proposal Writing: An Expert's Stres...,5943,59641
1031123,1931333246,10,Chorus,5943,110132
1031127,0312264186,8,You Can't Catch Death: A Daughter's Memoir,5944,14956


In [16]:
recommendations = recommendations[['BookID', 'Book-Title']]
recommendations = recommendations.reset_index()
recommendations

,index,BookID,Book-Title
0,1,21549,Flesh Tones: A Novel
1,66,42516,The Notebook
2,181,111065,Les Particules Elementaires
3,189,32359,A Painted House
4,370,36898,Lightning
...,...,...,...
119467,1031120,8527,Time Out Chicago (Time Out Chicago)
119468,1031121,59641,The Zen of Proposal Writing: An Expert's Stres...
119469,1031123,110132,Chorus
119470,1031127,14956,You Can't Catch Death: A Daughter's Memoir


In [17]:
recommendations = recommendations.drop(['index'], axis=1)
recommendations

,BookID,Book-Title
0,21549,Flesh Tones: A Novel
1,42516,The Notebook
2,111065,Les Particules Elementaires
3,32359,A Painted House
4,36898,Lightning
...,...,...
119467,8527,Time Out Chicago (Time Out Chicago)
119468,59641,The Zen of Proposal Writing: An Expert's Stres...
119469,110132,Chorus
119470,14956,You Can't Catch Death: A Daughter's Memoir


In [18]:
userid = user.values[0]

In [19]:
trainset = data.build_full_trainset()
svd.fit(trainset)
recommendations['Predicted-Rating'] = recommendations['BookID'].apply(lambda x: svd.predict(userid, x).est)
recommendations = recommendations.sort_values('Predicted-Rating', ascending=False)
recommendations.head(10)

,BookID,Book-Title,Predicted-Rating
25872,38769,Harry Potter and the Chamber of Secrets Postca...,9.870177
22197,73132,"My Sister's Keeper : A Novel (Picoult, Jodi)",9.845524
6579,19948,"The Return of the King (The Lord of the Rings,...",9.825329
12666,7643,84 Charing Cross Road,9.820471
5220,19947,"The Two Towers (The Lord of the Rings, Part 2)",9.747241
4844,59813,"The Two Towers (The Lord of the Rings, Part 2)",9.711544
48805,86366,Dilbert: A Book of Postcards,9.708438
837,3399,A Tree Grows in Brooklyn,9.707270
996,41269,Dune (Remembering Tomorrow),9.703754
4796,38487,Harry Potter and the Goblet of Fire (Book 4),9.694825
